In [1]:
import numpy as np
import torch
from torch.utils import data
from tqdm import tqdm

**1.生成数据集**

In [2]:
true_w = torch.tensor([2,-3.4])
true_b = 4.2

def create_data(w, b):
    X = torch.normal(0, 1, size=(1000, 2))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.view(-1,1)

features, labels = create_data(true_w, true_b)

**2. 加载数据集**

In [3]:
def load_array(data_array, batch_size, is_train = True):
    dataset = data.TensorDataset(*data_array)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [4]:
next(iter(data_iter))

[tensor([[-0.1609,  1.0997],
         [ 0.7617, -1.1926],
         [-0.8715,  2.1641],
         [-0.7983,  1.6132],
         [-0.4958, -0.1669],
         [ 3.1260,  0.3877],
         [-0.6776, -0.2122],
         [ 0.7672,  0.6536],
         [-0.6566, -0.3963],
         [ 1.1266, -0.7430]]),
 tensor([[ 0.1490],
         [ 9.7663],
         [-4.9121],
         [-2.8789],
         [ 3.7731],
         [ 9.1221],
         [ 3.5610],
         [ 3.5109],
         [ 4.2291],
         [ 8.9754]])]

**3. 定义模型**

In [5]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

**4. 初始化模型参数**

In [6]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)
# 带下划线的函数一般是原地操作，不能记录梯度

tensor([0.])

**5. 定义损失函数**

In [7]:
loss = nn.MSELoss()

**6. 定义优化算法**

In [8]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

**7. 训练**

In [9]:
num_epochs = 3
for i in tqdm(range(num_epochs)):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {i + 1}, loss {l:f}')

100%|██████████| 3/3 [00:00<00:00, 25.75it/s]

epoch 1, loss 0.000242
epoch 2, loss 0.000106
epoch 3, loss 0.000105


* `l.backward()` : 意味着，沿着计算图，反向计算期间各个参数的梯度

* `trainer.zero_grad()` : 意味着，将此时各个遍历的梯度置 0 

In [10]:
w = net[0].weight.data
b = net[0].bias.data
print('w的估计误差:', true_w - w.reshape(true_w.shape))
print('b的估计误差:', true_b - b)

w的估计误差: tensor([-4.1938e-04,  5.6982e-05])
b的估计误差: tensor([-0.0002])
